# Down and Out Call
### Basics
This notebook presents the pricing of a down and out call option. The first cells cover only a Naive Monte Carlo. Antithetic and stratified sampling are then introduced. Each simulation returns the option price, standard error, and simulation runtime. The hope is that imrpoved sampling would return a lower standard error without increasing runtime considerably. The last cell contains a table comparing each of the results.

You will find the standard error to be fairly high for most these simulations. This is due to a low number of steps (10) and simulations (100) in the model. While experimenting with higher simulation numbers (up to M = 1,000,000) we found the down and out call price should be close to 5.0 when using the Naive Monte Carlo. However, the stratified samples returned values closer to 1.6.

We also ran a pure European Binomial model with these same parameters. The result came out consistantly around 11.0. We then expect the results of the down and out model to be significantly less than 11.0.

## Naive Monte Carlo

In [36]:
import numpy as np
import time

spot = 100   
strike = 100
rate = 0.06
expiry = 1.0
volatility = 0.20
dividend = 0.03
N = 10
M = 100
H = 99

dt = expiry / N
nudt = (rate-dividend-0.5*volatility**2)*dt
sigdt = volatility * np.sqrt(dt)

running_price = 0
spot_t = np.empty((M,N))
spot_t[:,0] = spot
Ct = np.zeros(M)

t1 = time.time()
barrierCrossed = False

for i in range(M):
    z = np.random.normal(size = N)
    barrierCrossed = False
    for j in range(1, N):
        #spot_t[0] = spot
        spot_t[i,j] = spot_t[i,j-1] * np.exp(nudt + sigdt * z[j])
        if(spot_t[i,j] <= H):
            barrierCrossed = True
            break
    if not barrierCrossed:
        Ct[i] = np.maximum(spot_t[i,j]-strike, 0.0)

running_price = sum(Ct)
t2 = time.time()

se = np.std(Ct)/np.sqrt(M)

Payoff = running_price/M
PayoffPV = np.exp(-rate *expiry) * Payoff
print("The down and out Monte Carlo call payoff is", PayoffPV)
print("The Standard Error is", se)
print("The time to run the function is", (t2-t1))

The down and out Monte Carlo call payoff is 5.13724691231
The Standard Error is 1.21988423773
The time to run the function is 0.004000663757324219


We can modify this simple code to work for a put option as well. However, this barrier is still solely for a down and out option. The prices for both the call option and put option are listed below in the following object oriented code:

In [4]:
import numpy as np
import time

t1 = time.time()

class Option:
    def __init__(self, strike, expiry):
        self.strike = strike
        self.expiry = expiry

    def value(self):
        pass

class CallOption(Option):
    def value(self, spot):
        return np.maximum(0.0, spot - self.strike)

class PutOption(Option):
    def value(self, spot):
        return np.maximum(0.0, self.strike - spot)
            
def DownOut(option, spot, volatility, rate, dividend, M, N):
    Ct = np.zeros(M)
    H = 99
    spot_t = np.zeros((M,N))
    spot_t[:,0] = spot
    dt = option.expiry / N
    nudt = (rate-dividend-0.5*volatility**2)*dt
    sigdt = volatility * np.sqrt(dt)
    barrierCrossed = False

    for i in range(M):
        z = np.random.normal(size = N)
        barrierCrossed = False
        for j in range(1, N):
            spot_t[i,j] = spot_t[i,j-1] * np.exp(nudt + sigdt * z[j])
            if(spot_t[i,j] <= H):
                barrierCrossed = True
                break
            if not barrierCrossed:
                Ct[i] = option.value(spot_t[i,j])
    return Ct

spot = 100    
strike = 100
rate = 0.06
expiry = 1.0
volatility = 0.20
dividend = 0.03
M = 100
N = 10

theCall = CallOption(strike, expiry)
thePut = PutOption(strike, expiry)
PriceArrayCall = np.zeros((M,N))
PriceArrayCall = DownOut(theCall, spot, volatility, rate, dividend, M, N)
PriceArrayPut = np.zeros((M,N))
PriceArrayPut = DownOut(thePut, spot, volatility, rate, dividend, M, N)
CallPrice = sum((PriceArrayCall)/M) * np.exp(-rate *expiry)
PutPrice = sum((PriceArrayPut)/M) * np.exp(-rate *expiry)
se = np.std(PriceArrayCall)/np.sqrt(M)

t2 = time.time()

print("The down and out Monte Carlo call payoff is", CallPrice)
print("The down and out Monte Carlo put payoff is", PutPrice)
print("The standard error is", se)
print("The time to run the function is", (t2-t1))

The down and out Monte Carlo call payoff is 6.4729559571
The down and out Monte Carlo put payoff is 0.0269355737943
The standard error is 1.24390591294
The time to run the function is 0.015619754791259766


## Antithetic Sampling
We now introduce antithetic sampling, which will in effect double our sample size. This should, in theory, decrease our standard error as we approach the law of large numbers faster. Any increase in the standard error may be due to the relatively small draw size.

In [7]:
import numpy as np
import time

spot = 100   
strike = 100
rate = 0.06
expiry = 1.0
volatility = 0.20
dividend = 0.03
N = 10
M = 100
H = 99

dt = expiry / N
nudt = (rate-dividend-0.5*volatility**2)*dt
sigdt = volatility * np.sqrt(dt)

running_price = 0
spot_t = np.empty((M,N))
spot_t[:,0] = spot
Ct = np.zeros(M)

t1 = time.time()
barrierCrossed = False

for i in range(M):
    z1 = np.random.normal(size = N)
    z2 = -z1
    z = np.concatenate([z1,z2])
    barrierCrossed = False
    for j in range(1, N):
        #spot_t[0] = spot
        spot_t[i,j] = spot_t[i,j-1] * np.exp(nudt + sigdt * z[j])
        if(spot_t[i,j] <= H):
            barrierCrossed = True
            break
    if not barrierCrossed:
        Ct[i] = np.maximum(spot_t[i,j]-strike, 0.0)

running_price = sum(Ct)
t2 = time.time()

se = np.std(Ct)/np.sqrt(M)

Payoff = running_price/M
PayoffPV = np.exp(-rate *expiry) * Payoff
print("The down and out Monte Carlo call payoff is", PayoffPV)
print("The Standard Error is", se)
print("The time to run the function is", (t2-t1))

The down and out Monte Carlo call payoff is 4.32040188274
The Standard Error is 0.944690609044
The time to run the function is 0.008158683776855469


## Stratified Sampling

In [5]:
import numpy as np
from scipy.stats import norm
import math
import time

spot = 100   
strike = 100
rate = 0.06
expiry = 1.0
volatility = 0.20
dividend = 0.03
N = 16
M = 100
H = 99

endval=np.random.normal(loc=0.0, scale=np.sqrt(expiry), size=1)
dt = expiry / N
nudt = (rate-dividend-0.5*volatility**2)*dt
sigdt = volatility * np.sqrt(dt)

running_price = 0
spot_t = np.zeros((M,N))
spot_t[:,0] = spot
Ct = np.zeros(M)

t1 = time.time()

def StratifiedSample(M = 100):
    u = np.random.uniform(size=M)  
    i = np.arange(M)
    uhat = (u + i) / M

    return uhat

def WienerBridge(expiry, num_steps, endval = 0.0):
    num_bisect = int(math.log2(num_steps))
    tjump = int(expiry)
    ijump = int(num_steps - 1)

    if endval == 0.0:
        endval = np.random.normal(scale = np.sqrt(expiry), size=1)

    z = np.random.normal(size=num_steps + 1)
    w = np.zeros(num_steps + 1)
    w[num_steps] = endval
    

    for k in range(num_bisect):
        left = 0
        i = ijump // 2 + 1
        right = ijump + 1
        limit = 2 ** k

        for j in range(limit):
            a = 0.5 * (w[left] + w[right])
            b = 0.5 * np.sqrt(tjump)
            w[i] = a + b * z[i]
            right += ijump + 1
            left += ijump + 1
            i += ijump + 1
        
        ijump //= 2    
        tjump /= 2

    return np.diff(w) 

uhat = StratifiedSample(M)
endval = norm.ppf(uhat)

barrierCrossed = False

for i in range(M):
    z = WienerBridge(expiry, N, endval[i])
    #z = np.random.normal(size=N)
    barrierCrossed = False
    for j in range(1, N):
        spot_t[i,j] = spot_t[i,j-1] * np.exp(nudt + sigdt * z[j])
        if(spot_t[i,j] <= H):
            barrierCrossed = True
            break
    if not barrierCrossed:
        Ct[i] = np.maximum(spot_t[i,-1]-strike, 0.0)

running_price = np.sum(Ct)
t2 = time.time()

se = np.std(Ct)/np.sqrt(M)

Payoff = running_price / M
PayoffPV = np.exp(-rate *expiry) * Payoff
print("The down and out Monte Carlo call payoff is", PayoffPV)
print("The Standard Error is", se)
print("The time to run the function is", (t2-t1))

The down and out Monte Carlo call payoff is 1.33081674146
The Standard Error is 0.297318506117
The time to run the function is 0.015622615814208984


## Antithetic and Statified Sampling

In [6]:
import numpy as np
from scipy.stats import norm
import math
import time

spot = 100   
strike = 100
rate = 0.06
expiry = 1.0
volatility = 0.20
dividend = 0.03
N = 16
M = 100
H = 99

endval=np.random.normal(loc=0.0, scale=np.sqrt(expiry), size=1)
dt = expiry / N
nudt = (rate-dividend-0.5*volatility**2)*dt
sigdt = volatility * np.sqrt(dt)

running_price = 0
spot_t = np.zeros((M,N))
spot_t[:,0] = spot
Ct = np.zeros(M)

t1 = time.time()

def StratifiedSample(M = 100):
    u = np.random.uniform(size=M)  
    i = np.arange(M)
    uhat = (u + i) / M

    return uhat

def WienerBridge(expiry, num_steps, endval = 0.0):
    num_bisect = int(math.log2(num_steps))
    tjump = int(expiry)
    ijump = int(num_steps - 1)

    if endval == 0.0:
        endval = np.random.normal(scale = np.sqrt(expiry), size=1)

    z = np.random.normal(size=num_steps + 1)
    w = np.zeros(num_steps + 1)
    w[num_steps] = endval
    

    for k in range(num_bisect):
        left = 0
        i = ijump // 2 + 1    
        right = ijump + 1
        limit = 2 ** k

        for j in range(limit):
            a = 0.5 * (w[left] + w[right])
            b = 0.5 * np.sqrt(tjump)
            w[i] = a + b * z[i]
            right += ijump + 1
            left += ijump + 1
            i += ijump + 1
        
        ijump //= 2   
        tjump /= 2

    return np.diff(w)

uhat = StratifiedSample(M)
endval = norm.ppf(uhat)

barrierCrossed = False

for i in range(M):
    z1 = WienerBridge(expiry, N, endval[i])
    z2 = -z1
    z = np.concatenate([z1,z2])
    
    barrierCrossed = False
    for j in range(1, N):
        spot_t[i,j] = spot_t[i,j-1] * np.exp(nudt + sigdt * z[j])
        if(spot_t[i,j] <= H):
            barrierCrossed = True
            break
    if not barrierCrossed:
        Ct[i] = np.maximum(spot_t[i,-1]-strike, 0.0)

running_price = np.sum(Ct)
t2 = time.time()

se = np.std(Ct)/np.sqrt(M)

Payoff = running_price / M
PayoffPV = np.exp(-rate *expiry) * Payoff
print("The down and out Monte Carlo call payoff is", PayoffPV)
print("The Standard Error is", se)
print("The time to run the function is", (t2-t1))

The down and out Monte Carlo call payoff is 1.54652049763
The Standard Error is 0.313710253353
The time to run the function is 0.031227588653564453


# Results

|                   | Naive Monte Carlo | Antithetic Sampling | Stratified Sampling | Antithetic and Stratified Sampling |
|:-----------------:|:-----------------:|:-------------------:|:-------------------:|:----------------------------------:|
| **Call Option Price** |      5.137246     |       4.320401      |       1.330816      |              1.546520              |
|   **Standard Error**  |      1.219884     |       0.944690      |       0.297318      |              0.313710              |
|  **Computation Time** |      0.004000     |       0.008158      |       0.015622      |              0.031227              |

We can see that as we introduce increased sampling techniques, the computation time increases while the standard error decreases. This is consistant with our expectations. It is difficult to make an assesment of the marginal benefit of increased sampling acuracy. With only 100 simulations, we argue that the increased sampling accuracy is worth the relatively small increase in computation time. This may change as our sample draw is increased or the number of steps is increased.

We also noticed when stratified sampling was introduced, call price was significantly lower than the Naive Monte Carlo or the Atithetic model. While this could be a more accurate option price, we believe there may be some dsicrepancy in the code. 